# REF Python 3.12

## LIBS

In [2]:
import dis
import re

## CLASSES DEFINITIONS

In [86]:
class Stack:
    def __init__(self, size):
        self.stack = []
        self.size = size
        self.top = -1
        self.empty = True
        self.available_instructions = {
            1   : "POP_TOP",
            4   : "END_FOR",
            5   : "END_SEND",
            9   : "NOP",
            83  : "RETURN_VALUE",
            99  : "SWAP",
            120 : "COPY",
        }

    def SHOW_STACK(self):
        if self.empty:
            print("EMPTY STACK")
            print()
            return
        
        print("STACK:")
        for i in range(self.top, -1, -1):
            print(self.stack[i] if i != self.top else f"{self.stack[i]} <-- TOS")
        print("EOS")
        print()

    def NOP(self):
        pass

    def RETURN_VALUE(self):
        if self.empty:
            return None
        return self.stack[self.top]
    
    def PUSH(self, value):
        if self.top == self.size - 1:
            print("STACK OVERFLOW")
            return
        
        self.stack.append(value)
        self.top += 1
        self.empty = False

    def POP_TOP(self):
        if self.empty:
            print("STACK UNDERFLOW -- EMPTY STACK")
            return
        
        self.top -= 1
        if self.top == -1:
            self.empty = True

        return self.stack.pop()

    def END_FOR(self):
        for i in range(2):
            if self.empty:
                print("STACK UNDERFLOW -- EMPTY STACK" if i == 0 else "STACK UNDERFLOW -- ONE ELEMENT REMOVED")
                return
            
            self.stack.pop()
            self.top -= 1
            if self.top == -1:
                self.empty = True

    def END_SEND(self):
        if self.top < 1:
            print("STACK UNDERFLOW -- NOT ENOUGH ELEMENTS")
            return
        
        del self.stack[-2]
        self.top -= 1

    def COPY(self, index):
        if index < 0:
            print("INDEX MUST BE GREATER THAN -1")
            return 

        if self.top < index:
            print("STACK UNDERFLOW -- INDEX OUT OF RANGE")
            return
        
        self.PUSH(self.stack[-index])

    def SWAP(self, index):
        if index < 0:
            print("INDEX MUST BE GREATER THAN -1")
            return 

        if self.top < index:
            print("STACK UNDERFLOW -- INDEX OUT OF RANGE")
            return
        
        self.stack[self.top], self.stack[-index] = self.stack[-index], self.stack[self.top]

    def FETCH_POS(self, index):
        if index < 0 or index > self.top:
            print("INDEX OUT OF RANGE")
            return
        
        return self.stack[-index]

In [87]:
class ULA:
    def __init__(self, size):
        self.stack = Stack(size)
        
        self.available_instructions = {
            11  : "UNARY_NEGATIVE",
            12  : "UNARY_NOT",
            15  : "UNARY_INVERT",
            107 : "COMPARE_OP",
            117 : "IS_OP",
            118 : "CONTAINS_OP",
            122 : "BINARY_OP",
        }

        self.available_bin_operators = {
            0  : "+",
            1  : "&",
            2  : "//",
            3  : "<<",
            5  : "*",
            6  : "%",
            7  : "|",
            8  : "**",
            9  : ">>",
            10 : "-",
            11 : "/",
            12 : "^",
            13 : "+=",
            14 : "&=",
            15 : "//=",
            16 : "<<=",
            18 : "*=",
            19 : "%=",
            20 : "|=",
            21 : "**=",
            22 : ">>=",
            23 : "-=",
            24 : "/=",
            25 : "^=",  
        }

        self.available_cmp_operators = {
            2  : "<",
            26 : "<=",
            40 : "==",
            55 : "!=",
            68 : ">",
            92 : ">=",
        }

        self.available_identity_operators = {
            0 : "is",
            1 : "is not",
        }

        self.available_contains_operators = {
            0 : "in",
            1 : "not in",
        }

    def UNARY_NEGATIVE(self):
        tos = self.stack.POP_TOP()
        self.stack.PUSH(-tos)

    def UNARY_NOT(self):
        tos = self.stack.POP_TOP()
        self.stack.PUSH(not tos)

    def UNARY_INVERT(self):
        tos = self.stack.POP_TOP()
        self.stack.PUSH(~tos)
        
    def BINARY_OP(self, operator):
        if self.stack.top < 1:
            print("STACK UNDERFLOW -- NOT ENOUGH ELEMENTS")
            return
        
        if operator not in self.available_bin_operators:
            print("INVALID OPERATOR")
            return
        
        rhs = self.stack.POP_TOP()
        lhs = self.stack.POP_TOP()

        self.stack.PUSH(eval(f"{lhs} {self.available_bin_operators[operator]} {rhs}"))

    def COMPARE_OP(self, operator):
        if self.stack.top < 1:
            print("STACK UNDERFLOW -- NOT ENOUGH ELEMENTS")
            return
        
        if operator not in self.available_cmp_operators:
            print("INVALID OPERATOR")
            return
        
        rhs = self.stack.POP_TOP()
        lhs = self.stack.POP_TOP()

        self.stack.PUSH(eval(f"{lhs} {self.available_cmp_operators[operator]} {rhs}"))

    def IS_OP(self, operator):
        if self.stack.top < 1:
            print("STACK UNDERFLOW -- NOT ENOUGH ELEMENTS")
            return
        
        if operator not in self.available_identity_operators:
            print("INVALID OPERATOR")
            return
        
        rhs = self.stack.POP_TOP()
        lhs = self.stack.POP_TOP()

        self.stack.PUSH(eval(f"{lhs} {self.available_identity_operators[operator]} {rhs}"))

    def CONTAINS_OP(self, operator):
        if self.stack.top < 1:
            print("STACK UNDERFLOW -- NOT ENOUGH ELEMENTS")
            return
        
        if operator not in self.available_contains_operators:
            print("INVALID OPERATOR")
            return
        
        rhs = self.stack.POP_TOP()
        lhs = self.stack.POP_TOP()

        self.stack.PUSH(eval(f"{lhs} {self.available_contains_operators[operator]} {rhs}"))

In [88]:
class Memory:
  def __init__(self, instructions, constants, locals_var):
    self.instructions = instructions
    self.constants = constants
    self.locals = [{var: 0} for var in locals_var]

    self.available_instructions = {
      121 : "RETURN_CONST",
    }

    self.pc = 0

  def fetch(self):
    instr = self.instructions[self.pc]
    self.pc += 1

    return instr
  
  def fetch_constant(self, index):
    if index < 0 or index >= len(self.constants):
      print("INDEX OUT OF RANGE")
      return

    return self.constants[index]
  
  def fetch_local(self, index):
    if index < 0 or index >= len(self.locals):
      print("INDEX OUT OF RANGE")
      return
    
    for key in self.locals[index]:
      return self.locals[index][key]

  def fetch_pc(self):
    return self.pc
  
  def set_pc(self, pc):
    if pc < 0 or pc >= len(self.instructions):
      print("PC OUT OF RANGE")
      return
    self.pc = pc

  def set_local(self, index, value):
    if index < 0 or index >= len(self.locals):
      print("INDEX OUT OF RANGE")
      return
    
    for key in self.locals[index]:
      self.locals[index][key] = value

  def remove_local(self, index):
    if index < 0 or index >= len(self.locals):
      print("INDEX OUT OF RANGE")
      return

    for key in self.locals[index]:
      self.locals[index].pop(key)

  def RETURN_CONST(self, index):
    if index < 0 or index >= len(self.constants):
      print("INDEX OUT OF RANGE")
      return
    return self.constants[index]

In [89]:
class CPU:
  def __init__(self):
    with open("load.txt", "r") as f:
        lines = f.readlines()

    stack_size = int(lines[0].split("|")[1])
    constants = eval(lines[1].split("|")[1])
    locals_var = eval(lines[2].split("|")[1])
    instructions = eval(lines[3].split("|")[1])

    self.MEM = Memory(instructions, constants, locals_var)
    self.ULA = ULA(stack_size)

    self.available_instructions = {
      1   : "POP_TOP",
      4   : "END_FOR",
      5   : "END_SEND",
      9   : "NOP",
      11  : "UNARY_NEGATIVE",
      12  : "UNARY_NOT",
      15  : "UNARY_INVERT",
      68  : "GET_ITER",
      83  : "RETURN_VALUE",
      93  : "FOR_ITER",
      99  : "SWAP",
      100 : "LOAD_CONST",
      102 : "BUILD_TUPLE",
      103 : "BUILD_LIST",
      104 : "BUILD_SET",
      107 : "COMPARE_OP",
      110 : "JUMP_FORWARD",
      114 : "POP_JUMP_IF_FALSE",
      115 : "POP_JUMP_IF_TRUE",
      117 : "IS_OP",
      118 : "CONTAINS_OP",
      120 : "COPY",
      121 : "RETURN_CONST",
      122 : "BINARY_OP",
      124 : "LOAD_FAST",
      125 : "STORE_FAST",
      126 : "DELETE_FAST",
      128 : "POP_JUMP_IF_NOT_NONE",
      129 : "POP_JUMP_IF_NONE",
      133 : "BUILD_SLICE",
      134 : "JUMP_BACKWARD_NO_INTERRUPT",
      140 : "JUMP_BACKWARD",
      150 : "YIELD_VALUE",
      151 : "RESUME",
      157 : "BUILD_STRING",
      162 : "LIST_EXTEND",
      163 : "SET_UPDATE",
      260 : "JUMP",
      261 : "JUMP_NO_INTERRUPT",
    }

    # opcode -> (function, has_argument)
    self.redirect = {
      1 : (self.ULA.stack.POP_TOP, 0),
      4 : (self.ULA.stack.END_FOR, 0),
      5 : (self.ULA.stack.END_SEND, 0),
      9 : (self.ULA.stack.NOP, 0),
      11 : (self.ULA.UNARY_NEGATIVE, 0),
      12 : (self.ULA.UNARY_NOT, 0),
      15 : (self.ULA.UNARY_INVERT, 0),
      83 : (self.ULA.stack.RETURN_VALUE, 0),
      99 : (self.ULA.stack.SWAP, 1),
      100 : (self.LOAD_CONST, 1),
      107 : (self.ULA.COMPARE_OP, 1),
      110 : (self.JUMP_FORWARD, 1),
      114 : (self.POP_JUMP_IF_FALSE, 1),
      115 : (self.POP_JUMP_IF_TRUE, 1),
      117 : (self.ULA.IS_OP, 1),
      118 : (self.ULA.CONTAINS_OP, 1),
      120 : (self.ULA.stack.COPY, 1),
      121 : (self.MEM.RETURN_CONST, 1),
      122 : (self.ULA.BINARY_OP, 1),
      124 : (self.LOAD_FAST, 1),
      125 : (self.STORE_FAST, 1),
      126 : (self.DELETE_FAST, 1),
      128 : (self.POP_JUMP_IF_NOT_NONE, 1),
      129 : (self.POP_JUMP_IF_NONE, 1),
      134 : (self.JUMP_BACKWARD_NO_INTERRUPT, 1),
      140 : (self.JUMP_BACKWARD, 1),
      151 : (self.RESUME, 0),
      260 : (self.JUMP, 1),
      261 : (self.JUMP_NO_INTERRUPT, 1),
    }

  def SHOW_CPU(self):
    print("CPU:")
    
    print("INSTRUCTIONS:")
    for instr in self.MEM.instructions:
      print(f"{self.available_instructions[instr[0]] if instr[0] in self.available_instructions else instr[0]} -> OPCODE {instr[0]} | ARGUMENT {instr[1]}")
    print()

    print(f"PC -> {self.MEM.fetch_pc()}")
    print()
        
    print("MEMORY:")
    print(f"CONSTANTS -> {self.MEM.constants}")
    print(f"LOCALS -> {self.MEM.locals}")
    print()

    print("STACK:")
    self.ULA.stack.SHOW_STACK()

  def LOAD_CONST(self, index):
    self.ULA.stack.PUSH(self.MEM.fetch_constant(index))

  def LOAD_FAST(self, index):    
    self.ULA.stack.PUSH(self.MEM.fetch_local(index))

  def STORE_FAST(self, index):    
    self.MEM.set_local(index, self.ULA.stack.POP_TOP())

  def DELETE_FAST(self, index):    
    self.MEM.remove_local(index)

  def JUMP_FORWARD(self, offset):
    self.MEM.set_pc(self.MEM.fetch_pc() + offset - 1)

  def JUMP_BACKWARD(self, offset):
    self.MEM.set_pc(self.MEM.fetch_pc() - offset - 1)

  def JUMP_BACKWARD_NO_INTERRUPT(self, offset):
    self.MEM.set_pc(self.MEM.fetch_pc() - offset)

  def POP_JUMP_IF_TRUE(self, offset):
    if self.ULA.stack.RETURN_VALUE() == True:
      self.JUMP_FORWARD(offset)
    
    self.ULA.stack.POP_TOP()

  def POP_JUMP_IF_FALSE(self, offset):
    if self.ULA.stack.RETURN_VALUE() == False:
      self.JUMP_FORWARD(offset)
    
    self.ULA.stack.POP_TOP()

  def POP_JUMP_IF_NOT_NONE(self, offset):
    if self.ULA.stack.RETURN_VALUE() != None:
      self.JUMP_FORWARD(offset)
    
    self.ULA.stack.POP_TOP()

  def POP_JUMP_IF_NONE(self, offset):
    if self.ULA.stack.RETURN_VALUE() == None:
      self.JUMP_FORWARD(offset)
    
    self.ULA.stack.POP_TOP()

  def JUMP(self, offset):
    if offset < 0:
      self.JUMP_BACKWARD(-offset)
    else:
      self.JUMP_FORWARD(offset)

  def JUMP_NO_INTERRUPT(self, offset):
    if offset < 0:
      self.JUMP_BACKWARD(-offset)
    else:
      self.JUMP_FORWARD(offset)

  def RESUME(self):
    pass

  def RUN(self):
    while self.MEM.fetch_pc() < len(self.MEM.instructions):
      print("-" * 50)
      print()
      instr = self.MEM.fetch()
      opcode = instr[0]
      argument = instr[1]

      if opcode in self.redirect:
        function, has_argument = self.redirect[opcode]
        if has_argument:
          function(argument)
        else:
          function()
      else:
        print("INVALID OPCODE")
        break
    
      print("PC -> ", self.MEM.fetch_pc() - 1)
      print(f"{self.available_instructions[opcode] if opcode in self.available_instructions else opcode} -> OPCODE {opcode} | ARGUMENT {argument}")
      print()  

      print("MEMORY:")
      print(f"CONSTANTS -> {self.MEM.constants}")
      print(f"LOCALS -> {self.MEM.locals}")
      print()

      if opcode == 83:
        print("RETURN VALUE ->", self.ULA.stack.RETURN_VALUE())
        print()
        break
      elif opcode == 121:
        print("RETURN VALUE ->", self.MEM.RETURN_CONST(argument))
        print()
        break
      else:
        print("STACK:")
        self.ULA.stack.SHOW_STACK()
      print("-" * 50)

In [3]:
def disassemble(code):
    infos = dis.code_info(code)

    constants = re.search(r"Constants:\s*(.*?)(?=\n[A-Z]|\Z)", infos, re.DOTALL)

    if constants:
        constant_lines = constants.group(1).strip().split("\n")
        constant_values = [line.split(': ')[1] for line in constant_lines]
        constant_values = [eval(value) for value in constant_values]
    else:
        constant_values = []

    variables = re.search(r"Variable names:\s*(.*?)(?=\n[A-Z]|\Z)", infos, re.DOTALL)

    if variables:
        variable_lines = variables.group(1).strip().split("\n")
        variable_values = [line.split(': ')[1] for line in variable_lines]
    else:
        variable_values = []

    stack_size = re.search(r"Stack size:\s*(\d+)", infos).group(1)

    instructions = list(dis.get_instructions(code))

    reduced_instructions = [(instr.opcode, instr.arg) for instr in instructions]

    print(instructions)
    
    with open("load.txt", "w") as f:
        f.write(f"STACK SIZE|{stack_size}\n")
        f.write(f"CONSTANTS|{constant_values}\n")
        f.write(f"LOCALS|{variable_values}\n")
        f.write(f"INSTRUCTIONS|{reduced_instructions}\n")

    return instructions

In [4]:
def my_function():
    a=1
    b=2
    c=a+b
    return c

In [5]:
disassemble(my_function)

[Instruction(opname='RESUME', opcode=151, arg=0, argval=0, argrepr='', offset=0, starts_line=1, is_jump_target=False, positions=Positions(lineno=1, end_lineno=1, col_offset=0, end_col_offset=0)), Instruction(opname='LOAD_CONST', opcode=100, arg=1, argval=1, argrepr='1', offset=2, starts_line=2, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=6, end_col_offset=7)), Instruction(opname='STORE_FAST', opcode=125, arg=0, argval='a', argrepr='a', offset=4, starts_line=None, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=4, end_col_offset=5)), Instruction(opname='LOAD_CONST', opcode=100, arg=2, argval=2, argrepr='2', offset=6, starts_line=3, is_jump_target=False, positions=Positions(lineno=3, end_lineno=3, col_offset=6, end_col_offset=7)), Instruction(opname='STORE_FAST', opcode=125, arg=1, argval='b', argrepr='b', offset=8, starts_line=None, is_jump_target=False, positions=Positions(lineno=3, end_lineno=3, col_offset=4, end_col_offset

[Instruction(opname='RESUME', opcode=151, arg=0, argval=0, argrepr='', offset=0, starts_line=1, is_jump_target=False, positions=Positions(lineno=1, end_lineno=1, col_offset=0, end_col_offset=0)),
 Instruction(opname='LOAD_CONST', opcode=100, arg=1, argval=1, argrepr='1', offset=2, starts_line=2, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=6, end_col_offset=7)),
 Instruction(opname='STORE_FAST', opcode=125, arg=0, argval='a', argrepr='a', offset=4, starts_line=None, is_jump_target=False, positions=Positions(lineno=2, end_lineno=2, col_offset=4, end_col_offset=5)),
 Instruction(opname='LOAD_CONST', opcode=100, arg=2, argval=2, argrepr='2', offset=6, starts_line=3, is_jump_target=False, positions=Positions(lineno=3, end_lineno=3, col_offset=6, end_col_offset=7)),
 Instruction(opname='STORE_FAST', opcode=125, arg=1, argval='b', argrepr='b', offset=8, starts_line=None, is_jump_target=False, positions=Positions(lineno=3, end_lineno=3, col_offset=4, end_col_of

In [95]:
c = CPU()


In [96]:
c.RUN()

--------------------------------------------------

PC ->  0
RESUME -> OPCODE 151 | ARGUMENT 0

MEMORY:
CONSTANTS -> [None, 1, 2]
LOCALS -> [{'a': 0}, {'b': 0}, {'c': 0}]

STACK:
EMPTY STACK

--------------------------------------------------
--------------------------------------------------

PC ->  1
LOAD_CONST -> OPCODE 100 | ARGUMENT 1

MEMORY:
CONSTANTS -> [None, 1, 2]
LOCALS -> [{'a': 0}, {'b': 0}, {'c': 0}]

STACK:
STACK:
1 <-- TOS
EOS

--------------------------------------------------
--------------------------------------------------

PC ->  2
STORE_FAST -> OPCODE 125 | ARGUMENT 0

MEMORY:
CONSTANTS -> [None, 1, 2]
LOCALS -> [{'a': 1}, {'b': 0}, {'c': 0}]

STACK:
EMPTY STACK

--------------------------------------------------
--------------------------------------------------

PC ->  3
LOAD_CONST -> OPCODE 100 | ARGUMENT 2

MEMORY:
CONSTANTS -> [None, 1, 2]
LOCALS -> [{'a': 1}, {'b': 0}, {'c': 0}]

STACK:
STACK:
2 <-- TOS
EOS

-------------------------------------------------

### ORDEM DE EXECUCAO

1. CRIE SUA FUNCAO
-- def my_function():

2. DISASSEMBLE SUA FUNCAO
-- disassemble(my_function)

3. CRIE O OBJETO CPU
-- cpu = CPU()

4. RODE O PROGRAMA
-- cpu.RUN()

### LISTS, SETS, DICTS, TUPLES, ITERATORS AND GENERATORS
  * BINARY_SUBSCR
  * STORE_SUBSCR
  * DELETE_SUBSCR
  * BINARY_SLICE
  * STORE_SLICE
  * SET_ADD
  * LIST_APPEND
  * MAP_ADD
  * YIELD_VALUE
  * GET_LEN
  * MATCH_MAPPING
  * MATCH_SEQUENCE
  * MATCH_KEYS
  * UNPACK_SEQUENCE
  * UNPACK_EX
  * BUILD_TUPLE
  * BUILD_LIST
  * BUILD_SET
  * BUILD_MAP
  * BUILD_CONST_KEY_MAP
  * LIST_EXTEND
  * SET_UPDATE
  * DICT_UPDATE
  * DICT_MERGE
  * CONTAINS_OP
  * FOR_ITER

In [94]:
  # def GET_ITER(self):
  #   tos = self.ULA.stack.POP_TOP()
  #   self.ULA.stack.PUSH(iter(tos))

  # def GET_YIELD_FROM_ITER(self):
  #   tos = self.ULA.stack.POP_TOP()
  #   if isinstance(tos, iter):
  #     self.ULA.stack.PUSH(tos)
  #   else:
  #     self.ULA.stack.PUSH(iter(tos))

  # def YIELD_VALUE(self):
  #   if isinstance(self.ULA.stack.RETURN_VALUE(), iter):
  #     self.ULA.stack.POP_TOP()
  #   else:
  #     print("NOT AN ITERATOR")

  # def GET_LEN(self):
  #   tos = self.ULA.stack.RETURN_VALUE()

  #   if isinstance(tos, (list, tuple, dict, str, set)):
  #     self.ULA.stack.PUSH(len(tos))
  #   else:
  #     print("NOT A SEQUENCE")

  # def BUILD_LIST(self, count):
  #   elements = []
  #   for _ in range(count):
  #     elements.append(self.ULA.stack.POP_TOP())
    
  #   self.ULA.stack.PUSH(elements)

  # def BUILD_TUPLE(self, count):
  #   elements = []
  #   for _ in range(count):
  #     elements.append(self.ULA.stack.POP_TOP())
    
  #   self.ULA.stack.PUSH(tuple(elements))

  # def BUILD_SET(self, count):
  #   elements = set()
  #   for _ in range(count):
  #     elements.add(self.ULA.stack.POP_TOP())
    
  #   self.ULA.stack.PUSH(elements)

  # def BUILD_STRING(self, count):
  #   elements = []
  #   for _ in range(count):
  #     elements.append(self.ULA.stack.POP_TOP())
    
  #   self.ULA.stack.PUSH("".join(elements))

  # def LIST_EXTEND(self, index):
  #   seq = self.ULA.stack.POP_TOP()
  #   if not isinstance(seq, list):
  #     print("NOT A LIST")
  #     return
    
  #   ext = self.ULA.stack.FETCH_POS(index)
  #   if not isinstance(ext, list):
  #     print("NOT A LIST")
  #     return
    
  #   self.ULA.stack.PUSH(list.extend(ext, seq))
    
  # def SET_UPDATE(self, index):
  #   seq = self.ULA.stack.POP_TOP()
  #   if not isinstance(seq, set):
  #     print("NOT A SET")
  #     return
    
  #   ext = self.ULA.stack.FETCH_POS(index)
  #   if not isinstance(ext, set):
  #     print("NOT A SET")
  #     return
    
  #   self.ULA.stack.PUSH(set.update(ext, seq))

  # def BUILD_SLICE(self, argc):
  #   if argc == 2:
  #     end = self.ULA.stack.POP_TOP()
  #     start = self.ULA.stack.POP_TOP()
  #     self.ULA.stack.PUSH(slice(start, end))
  #   elif argc == 3:
  #     step = self.ULA.stack.POP_TOP()
  #     end = self.ULA.stack.POP_TOP()
  #     start = self.ULA.stack.POP_TOP()
  #     self.ULA.stack.PUSH(slice(start, end, step))
  #   else:
  #     print("INVALID ARGUMENT COUNT")
    
  # def FOR_ITER(self, delta):
  #   tos = self.ULA.stack.POP_TOP()
  #   if isinstance(tos, iter):
  #     try:
  #       self.ULA.stack.PUSH(next(tos))
  #     except StopIteration:
  #       self.MEM.set_pc(self.MEM.fetch_pc() + delta - 1)
